In [17]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [28]:
with open("key.txt", "r") as file:
    key = file.read()

In [29]:
key = key.strip("\n")

In [182]:
# empty dict of businesses with key "name", "rating", "zip code"
restaurants = {"name": [], "rating":[], "zip_code": []}


In [183]:
offset = 0
while offset <= 950:
    url = f"https://api.yelp.com/v3/businesses/search?location=berlin&term=food&categories=italian%2C%20pizza&limit=50&offset={offset}"

    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {key}"
    }
    response = requests.get(url, headers=headers)
    #print(response.status_code)
    
    resp_dict = response.json()
    
    for ent in resp_dict["businesses"]:
        restaurants["name"].append(ent["alias"])
        restaurants["rating"].append(ent["rating"])
        restaurants["zip_code"].append(ent["location"]["zip_code"])
    offset += 50

In [184]:
df_rest = pd.DataFrame.from_dict(restaurants)

In [185]:
df_rest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      1000 non-null   object 
 1   rating    1000 non-null   float64
 2   zip_code  1000 non-null   object 
dtypes: float64(1), object(2)
memory usage: 23.6+ KB


In [113]:
url = "https://www.berlinstadtservice.de/xinh/Postleitzahlen_Berlin_Alphabetisch.html"
resp_scrape = requests.get(url)

soup = BeautifulSoup(resp_scrape.content, "html.parser")

In [114]:
table_entries = soup.find_all("td")

In [115]:
zip_codes = []

In [116]:
for ent in table_entries:
    zip_codes.append(ent.text)

In [117]:
clean_zip = []

In [118]:
for ent in zip_codes:
    ent = ent.replace("B-", "")
    clean_zip.append(ent)

In [199]:
plz = []
localities = []
for ent in clean_zip:
    plz.append(ent.split()[0])
    localities.append(" ".join(ent.split()[1:]))

In [47]:
wiki_resp = requests.get("https://en.wikipedia.org/wiki/Boroughs_and_neighborhoods_of_Berlin")
wiki_soup = BeautifulSoup(wiki_resp.content, "html.parser")

In [48]:
first_dl = wiki_soup.find("dl")

In [49]:
first_dl

<dl><dt>(01) <a href="/wiki/Mitte" title="Mitte">Mitte</a></dt></dl>

In [59]:
first_dl.find_next("tbody").find_next("tr").find_next("tr").find_next("a").find_next("a")

<a href="/wiki/Mitte_(locality)" title="Mitte (locality)">Mitte</a>

In [60]:
### Given a table row tr, we find the "name" under find_next("a").find_next("a"); 
### we find the population under find_next("td").find_next("td").find_next("td")
#Let's try it out for Mitte
name = first_dl.find_next("tbody").find_next("tr").find_next("tr").find_next("a").find_next("a").text
population = first_dl.find_next("tbody").find_next("tr").find_next("tr").find_next("td").find_next("td").find_next("td").text

In [61]:
name

'Mitte'

In [62]:
population

'79,582\n'

In [85]:
names = []
populations = []

In [75]:
# look for dl then find_next(tbody) and find_next("tr") to ignore the header and store in cur_table.
# cur_row is then always one find_next("tr") more from wherever we are and see above for access to info inside of cur_row
cur_table = wiki_soup.find("dl").find_next("tbody").find_next("tr")
cur_row = cur_table.find_next("tr")
i =  0
for i in range(6):
    #print(cur_row.find_next("a").find_next("a").text)
    names.append(cur_row.find_next("a").find_next("a").text)
    #print(cur_row.find_next("td").find_next("td").find_next("td").text)
    populations.append(cur_row.find_next("td").find_next("td").find_next("td").text)
    cur_row = cur_row.find_next("tr")
    i += i

In [76]:
names

['Mitte', 'Moabit', 'Hansaviertel', 'Tiergarten', 'Wedding', 'Gesundbrunnen']

In [77]:
populations

['79,582\n', '69,425\n', '5,889\n', '12,486\n', '76,363\n', '82,729\n']

In [78]:
### let's try it at scale
all_dls = wiki_soup.find_all("dl")

In [88]:
lst_of_name_pop = []
for dl in all_dls:
    cur_table = dl.find_next("tbody").find_next("tr")
    #print(cur_table)
    cur_row = cur_table.find_next("tr")
    #print(cur_row)
    i = 0
    for i in range(15):
        pop = cur_row.find_next("td").find_next("td").find_next("td").text
        name = cur_row.find_next("a").find_next("a").text
        
        name_pop = (name, pop)
        lst_of_name_pop.append(name_pop)
        #names.append(cur_row.find_next("a").find_next("a").text)
        #populations.append(cur_row.find_next("td").find_next("td").find_next("td").text)
        i += 1
        cur_row = cur_row.find_next("tr")

<tr>
<td style="border-bottom:1px solid gray;"><span typeof="mw:File"><a class="mw-file-description" href="/wiki/File:DEU_Berlin-Mitte_(district)_COA.svg"><img class="mw-file-element" data-file-height="610" data-file-width="416" decoding="async" height="22" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/b9/DEU_Berlin-Mitte_%28district%29_COA.svg/15px-DEU_Berlin-Mitte_%28district%29_COA.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/b9/DEU_Berlin-Mitte_%28district%29_COA.svg/23px-DEU_Berlin-Mitte_%28district%29_COA.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/b9/DEU_Berlin-Mitte_%28district%29_COA.svg/30px-DEU_Berlin-Mitte_%28district%29_COA.svg.png 2x" width="15"/></a></span> (0101) <a href="/wiki/Mitte_(locality)" title="Mitte (locality)">Mitte</a>
</td>
<td style="border-bottom:1px solid gray; text-align:right;">10.70
</td>
<td style="border-bottom:1px solid gray; text-align:right;">79,582
</td>
<td style="border-bottom:1px solid gray; text

In [89]:
lst_of_name_pop

[('Mitte', '79,582\n'),
 ('Moabit', '69,425\n'),
 ('Hansaviertel', '5,889\n'),
 ('Tiergarten', '12,486\n'),
 ('Wedding', '76,363\n'),
 ('Gesundbrunnen', '82,729\n'),
 ('Friedrichshain', '114,050\n'),
 ('Friedrichshain', '114,050\n'),
 ('Kreuzberg', '147,227\n'),
 ('Prenzlauer Berg', '142,319\n'),
 ('Prenzlauer Berg', '142,319\n'),
 ('Weißensee', '45,485\n'),
 ('Blankenburg', '6,550\n'),
 ('Heinersdorf', '6,580\n'),
 ('Karow', '18,258\n'),
 ('Friedrichshain', '114,050\n'),
 ('Kreuzberg', '147,227\n'),
 ('Prenzlauer Berg', '142,319\n'),
 ('Prenzlauer Berg', '142,319\n'),
 ('Weißensee', '45,485\n'),
 ('Blankenburg', '6,550\n'),
 ('Heinersdorf', '6,580\n'),
 ('Karow', '18,258\n'),
 ('Stadtrandsiedlung Malchow', '1,166\n'),
 ('Pankow', '55,854\n'),
 ('Blankenfelde', '1,917\n'),
 ('Buch', '13,188\n'),
 ('Französisch Buchholz', '18,766\n'),
 ('Niederschönhausen', '26,903\n'),
 ('Rosenthal', '8,933\n'),
 ('Prenzlauer Berg', '142,319\n'),
 ('Weißensee', '45,485\n'),
 ('Blankenburg', '6,550\n'),

In [90]:
unique_name_pop = set(lst_of_name_pop)

In [91]:
unique_name_pop

{('Adlershof', '15,112\n'),
 ('Alt-Hohenschönhausen',
  '\nGesundbrunnen\nHansaviertel\nMitte\nMoabit\nTiergarten\nWedding\n'),
 ('Alt-Hohenschönhausen', '41,780\n'),
 ('Alt-Treptow', '10,426\n'),
 ('Altglienicke', '26,101\n'),
 ('Baumschulenweg', '16,780\n'),
 ('Biesdorf', '24,543\n'),
 ('Blankenburg', '6,550\n'),
 ('Blankenfelde', '1,917\n'),
 ('Bohnsdorf', '10,751\n'),
 ('Borsigwalde', '6,432\n'),
 ('Britz', '38,334\n'),
 ('Buch', '13,188\n'),
 ('Buckow', '38,018\n'),
 ('Charlottenburg',
  '\nAlt-Hohenschönhausen\nFalkenberg\nFennpfuhl\nFriedrichsfelde\nKarlshorst\nLichtenberg\nMalchow\nNeu-Hohenschönhausen\nRummelsburg\nWartenberg\n'),
 ('Charlottenburg', '118,704\n'),
 ('Charlottenburg-Nord', '17,327\n'),
 ('Dahlem', '14,966\n'),
 ('Falkenberg', '1,164\n'),
 ('Falkenhagener Feld', '34,778\n'),
 ('Fennpfuhl', '30,932\n'),
 ('Französisch Buchholz', '18,766\n'),
 ('Friedenau', '26,736\n'),
 ('Friedrichsfelde', '50,010\n'),
 ('Friedrichshagen', '17,285\n'),
 ('Friedrichshain',
  '\nBi

In [96]:
names = []
pops = []

In [92]:
names_pops = list(unique_name_pop)

In [100]:
for tup in names_pops:
    
    try:
        float(tup[1].replace("\n", "").replace(",", ""))
    except:
        continue
    else:
        names.append(tup[0])
        pops.append(tup[1])

In [104]:
print(names[-1], pops[-1])

Neukölln 154,127



In [105]:
len(names)

97

In [108]:
pops = [float(pop.replace("\n", "").replace(",", "")) for pop in pops]

In [110]:
pop_by_locality = {"name": names, "population": pops}

In [111]:
df_pops_by_locality = pd.DataFrame.from_dict(pop_by_locality)

In [112]:
df_pops_by_locality.head()

,name,population
0,Rosenthal,8933.0
1,Frohnau,17025.0
2,Französisch Buchholz,18766.0
3,Neu-Hohenschönhausen,53698.0
4,Mariendorf,48882.0


In [200]:
### df from localities and postcodes
df_zip_local = pd.DataFrame.from_dict({"locality": localities, "zip_code": plz})

In [124]:
df_zip_local.head()

,locality,zip_code
0,Adlershof,12487
1,Adlershof,12489
2,Altglienicke,12524
3,Alt-Treptow,12435
4,Baumschulenweg,12437


In [125]:
df_rest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      20 non-null     object 
 1   rating    20 non-null     float64
 2   zip_code  20 non-null     object 
dtypes: float64(1), object(2)
memory usage: 608.0+ bytes


In [225]:
df_restaurants = df_rest.merge(df_zip_local, on= "zip_code", how= "inner")

In [209]:
df_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1495 entries, 0 to 1494
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      1495 non-null   object 
 1   rating    1495 non-null   float64
 2   zip_code  1495 non-null   object 
 3   locality  1495 non-null   object 
dtypes: float64(1), object(3)
memory usage: 58.4+ KB


In [203]:
df_restaurants.tail()

,name,rating,zip_code,locality
1490,ristorante-al-castagno-berlin,4.0,13505,Tegel
1491,ristoranto-capitano-berlin-2,5.0,13435,Wittenau
1492,ristorante-siena-berlin,3.5,13158,Französisch Buchholz
1493,ristorante-siena-berlin,3.5,13158,Niederschönhausen
1494,ristorante-la-merita-berlin-2,3.0,12355,Rudow


In [191]:
df_rest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      1000 non-null   object 
 1   rating    1000 non-null   float64
 2   zip_code  1000 non-null   object 
dtypes: float64(1), object(2)
memory usage: 23.6+ KB


In [210]:
import numpy as np

In [211]:
np.setdiff1d(df_zip_local.locality, df_pops_by_locality.name)

array(['Berlin-Flughafen Schönefeld', 'Berlin-Karow', 'Berlin-Staaken',
       'Buchholz', 'Hessenwinkel', 'Hohenschönhausen', 'Karolinenhof',
       'Margaretenhöhe', 'Pichelsdorf', 'Wilhelmshagen'], dtype=object)

In [212]:
np.setdiff1d(df_pops_by_locality.name, df_zip_local.locality)

array(['Alt-Hohenschönhausen', 'Charlottenburg-Nord',
       'Falkenhagener Feld', 'Fennpfuhl', 'Friedrichsfelde',
       'Gesundbrunnen', 'Gropiusstadt', 'Halensee', 'Hansaviertel',
       'Haselhorst', 'Karow', 'Kladow', 'Moabit', 'Märkisches Viertel',
       'Neu-Hohenschönhausen', 'Schlachtensee', 'Staaken',
       'Stadtrandsiedlung Malchow', 'Wartenberg', 'Westend',
       'Wilhelmsruh', 'Wilhelmstadt'], dtype=object)

In [213]:
len(df_pops_by_locality.name)

97

In [214]:
len(df_zip_local.locality)

313

In [216]:
df_pops_by_locality.locality = df_pops_by_locality.name

/tmp/ipykernel_24421/2101200091.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_pops_by_locality.locality = df_pops_by_locality.name


In [217]:
df_pops_by_locality["locality"] = df_pops_by_locality.name

In [220]:
df_pops_by_locality = df_pops_by_locality.drop(columns=["name"])

In [221]:
df_restaurants = df_restaurants.merge(df_pops_by_locality, how= "inner", on="locality")

In [222]:
df_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1467 entries, 0 to 1466
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        1467 non-null   object 
 1   rating      1467 non-null   float64
 2   zip_code    1467 non-null   object 
 3   locality    1467 non-null   object 
 4   population  1467 non-null   float64
dtypes: float64(2), object(3)
memory usage: 68.8+ KB


In [226]:
top_rated = df_restaurants.loc[df_restaurants.rating >= 4, ]

In [229]:
agg_rest = top_rated.groupby(by="locality").count().reset_index()

In [231]:
per_capita = agg_rest.merge(df_pops_by_locality, how="inner", on = "locality")

In [241]:
per_capita["per_cap"] = per_capita.population/per_capita.name

In [245]:
per_capita = per_capita.sort_values(by="per_cap", ascending=False)

In [246]:
per_capita.head()

,locality,name,rating,zip_code,population,per_cap
39,Marzahn,1,1,1,102398.0,102398.000000
22,Hellersdorf,1,1,1,72602.0,72602.000000
2,Altglienicke,1,1,1,26101.0,26101.000000
4,Biesdorf,1,1,1,24543.0,24543.000000
29,Köpenick,3,3,3,59201.0,19733.666667


In [250]:
df_restaurants.loc[df_restaurants.zip_code.isin(["12679", "12681", "12685", "12687", "12689"]), ]

,name,rating,zip_code,locality
1340,antica-pizzicheria-berlin,3.5,12681,Marzahn
1418,i-fratelli-berlin,4.0,12685,Marzahn
1487,marzano-berlin,3.5,12687,Marzahn


In [249]:
df_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1495 entries, 0 to 1494
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      1495 non-null   object 
 1   rating    1495 non-null   float64
 2   zip_code  1495 non-null   object 
 3   locality  1495 non-null   object 
dtypes: float64(1), object(3)
memory usage: 58.4+ KB
